In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#LHCb. DataPreprocessor.
This class prepare LHCb data for other classes which are use used data in special form.

LHCb data is shown below:

In [2]:
lhcb_data = pd.read_excel('Data/popularity-728days.xls')
lhcb_data.irow(range(0,3))

,Name,Configuration,ProcessingPass,FileType,Type,Creation-week,NbLFN,LFNSize,NbDisk,DiskSize,...,95,96,97,98,99,100,101,102,103,104
0,/LHCb/Collision10/Beam3500GeV-VeloClosed-MagDo...,/LHCb/Collision10,/RealData/Reco08-MINBIAS-FIRST-14-NB/Stripping12d,MINIBIAS.DST,1,104591,67,0.317900,134,0.635799,...,12687,12687,12687,12687,12687,12687,12687,12687,12687,12687
1,/LHCb/Collision10/Beam3500GeV-VeloClosed-MagDo...,/LHCb/Collision10,/RealData/Reco08/Stripping14,BHADRON.DST,1,104607,227,0.649204,980,2.819260,...,1631,1631,1631,1631,1631,1631,1631,1631,1631,1631
2,/LHCb/Collision10/Beam3500GeV-VeloClosed-MagDo...,/LHCb/Collision10,/RealData/Reco08/Stripping14,CALIBRATION.DST,1,104607,355,1.370105,355,1.370105,...,0,0,0,474,708,708,708,708,708,708


LHCb data preparation and data form are shown below.

In [3]:
from datapop.LHCb import DataPreprocessor
data = DataPreprocessor(data=lhcb_data).get_preprocessed_data()
data.irow(range(0,3))

,ID,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,/LHCb/Collision10/Beam3500GeV-VeloClosed-MagDo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,134
1,/LHCb/Collision10/Beam3500GeV-VeloClosed-MagDo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,/LHCb/Collision10/Beam3500GeV-VeloClosed-MagDo...,0,0,0,0,0,234,474,0,0,...,0,0,0,0,0,0,0,0,0,0


#Core. ProbabilityEstimator.

This class is used to estimate dataset probabilities on given type of data storage. 
Higher a dataset's probability for a storage type, more suitable this storage type for the dataset in future.

The class accept the data form shown above.

In [ ]:
from datapop.core import ProbabilityEstimator
probability_estimator = ProbabilityEstimator(data=data, forecast_horizont=2, class_abs_thresholds=1)
probability_report = probability_estimator.get_probabilities()
probability_report.irow(range(0,3))

Also it is possible to get data that was used for the estimator training.

In [ ]:
probability_estimator.preprocessed_data.irow(range(0,3))

Furthermore, the class gives trained classifier which was used for probabilities estimation.

In [ ]:
classifier = probability_estimator.classifier

Some usefull metrics and pictures can be done by using probability_report

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(15, 10))

plt.subplot(2,2,1)
fpr, tpr, _ = roc_curve(probability_report['Current_Type'].values, probability_report['Proba_Type_1'].values)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='ROC (%.2f)' % roc_auc, c='r')
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='best')

plt.subplot(2,2,2)
plt.hist(probability_report['Proba_Type_1'].values, color='b', label='Proba_Type_1')
plt.title('Probability Type 1 distribution')
plt.xlabel('Probability')
plt.ylabel('Number of datasets')
plt.legend(loc='best')

plt.subplot(2,2,4)
cur_type = probability_report['Current_Type']
plt.hist(probability_report['Proba_Type_1'][cur_type==1].values, alpha=0.5, color='b', label='Current_Type_1')
plt.hist(probability_report['Proba_Type_1'][cur_type==0].values, alpha=0.5, color='r', label='Current_Type_0')
plt.title('Probability Type 1 distribution')
plt.xlabel('Probability')
plt.ylabel('Number of datasets')
plt.legend(loc='best')